# Testing of features in polymerist

In [ ]:
# Supressing annoying warnings (!must be done first!)
import warnings
warnings.catch_warnings(record=True)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

# Logging
from polymerist.genutils.logutils.IOHandlers import LOG_FORMATTER

import logging
logging.basicConfig(
    level=logging.INFO,
    format =LOG_FORMATTER._fmt,
    datefmt=LOG_FORMATTER.datefmt,
    force=True
)
LOGGER = logging.getLogger(__name__)

# General
import re, json
from pathlib import Path
from shutil import copyfile

import numpy as np
import matplotlib.pyplot as plt

# Logging
from rich.progress import Progress, track
import logging

# Chemistry
from openmm.unit import nanometer, angstrom
from openff.toolkit import Topology, Molecule, ForceField
from openff.units import unit as offunit

from openff.interchange import Interchange
from openff.interchange.components import _packmol as packmol

from rdkit import Chem
import openeye

# Custom
import polymerist as ps2
from polymerist.genutils.decorators.functional import allow_string_paths, allow_pathlib_paths, optional_in_place

## Testing combinatorics utils

In [ ]:
from polymerist.genutils import iteration
from polymerist.maths.combinatorics import core, graphenum, sequences, partitions

In [ ]:
for n in range(20):
    print(*(core.binomial_coeff(n, k) for k in range(n + 1)), sep=' ')

In [ ]:
for n in range(20):
    print(*(round(core.stirling_first(n, k)) for k in range(n + 1)), sep=' ')

In [ ]:
for n in range(20):
    print(round(core.bell_number(n)))

In [ ]:
for n in range(10):
    print(n, iteration.iter_len(partitions.int_partitions(n)))

In [ ]:
for n in range(20):
    print(round(core.catalan_number(n)))

# Testing OpenFF NAGL models

In [ ]:
from openff.toolkit import Molecule

from polymerist.mdtools.openfftools import TKREGS, TKWRAPPERS, NAGL_MODEL
from polymerist.mdtools.openfftools.pcharge import EspalomaCharger, NAGLCharger

from polymerist.rdutils.rdkdraw import rdmol_prop_heatmap_colorscaled
from polymerist.rdutils.rdprops import difference_rdmol

smi = 'CCCCCCC1=CSC=C1' # 3-hexylthiophene
# smi = 'OC(=O)CCCCC(=O)O' # adipic acid
# smi = 'O=C1OC(=O)c2ccccc12' # phthalic anhydride
# smi = 'CC(C)(c1ccc(cc1)O)c2ccc(cc2)O'# bisphenol A
mol = Molecule.from_smiles(smi)
mol.generate_conformers(toolkit_registry=TKREGS['OpenEye Toolkit'], n_conformers=1)
mol.visualize(backend='nglview')

In [ ]:
esp_mol = EspalomaCharger().charge_molecule(mol)
esp_mol.partial_charges

In [ ]:

nagl_mol = NAGLCharger().charge_molecule(mol)
nagl_mol.partial_charges

In [ ]:
import matplotlib.pyplot as plt
from rdkit import Chem

plt.plot(esp_mol.partial_charges - nagl_mol.partial_charges)

diff = difference_rdmol(nagl_mol.to_rdkit(), esp_mol.to_rdkit())
Chem.rdDepictor.Compute2DCoords(diff)
fig, ax = rdmol_prop_heatmap_colorscaled(diff, prop='DeltaPartialCharge')

# Experimenting with SDF files

In [ ]:
benz = Chem.MolFromSmiles('C1ccccC=1')
benz = Chem.AddHs(benz)
benz.SetDoubleProp('stuff', 3.14)
benz

In [ ]:
block2k = Chem.MolToMolFile(benz, 'test_2k.sdf')
block3k = Chem.MolToV3KMolFile(benz, 'test_3k.sdf')
block2kforce = Chem.MolToMolFile(benz, 'test_2k_force.sdf', forceV3000=True)

In [ ]:
with Chem.SDWriter('test_sdw.sdf') as sdwriter:
    sdwriter.SetForceV3000(True)
    print(sdwriter.GetForceV3000())

    sdwriter.write(benz)

In [ ]:
with Chem.SDMolSupplier('sdf_testing/test_off_rd.sdf', sanitize=False) as suppl:
    mols = [mol for mol in suppl]

targ = mols[0]
targ

In [ ]:
omol = Molecule.from_rdkit(benz)
omol.generate_conformers(n_conformers=1)
omol.visualize(backend='nglview')